In [1]:
from ginga import __version__
__version__

'3.0.1'

In [2]:
# import ginga remote control module
from ginga.util import grc

In [23]:
import pandas as pd
import re
import numpy as np

In [4]:
import lsst.daf.persistence as dafPersist
from pfs.lam.opdb import *


In [5]:
# host and port parameters can be set in the graphical ui of the plugin.
# These are the defaults
viewer = grc.RemoteClient('localhost', 9000)
#viewer = grc.RemoteClient('ws1.pfs', 9001)

In [6]:
viewer.host

'localhost'

In [7]:


# This gives you a handle to the reference viewer shell.
# It is similar to what you can do from the command line with:
#    $ grc ginga ...
#
shell = viewer.shell()

# the RC plugin in the viewer
shell.add_channel('Peak')
# This gives you a handle to a reference viewer channel.
# It is similar to what you can do from the command line with:
#    $ grc channel ...
#
ch = viewer.channel('Peak')

# Define data to be loaded

In [10]:
experimentId = 190 
rerun = "ws_slit"
#rerun = "dcb2"

repo = "sps"
cam = "r3"
arm = cam[0]
drpPath = "/data/drp"
basePath = '/drp/analysis/sm2/'

peaklist = "Imqual_LAM_peaklist_DEBUG_R3_Jan2020_WS_slit_2.csv"

In [11]:
visitStart, visitEnd = getVisitRange(experimentId)

In [12]:
visit = int(visitStart)

In [13]:
# define defaut parameters
roi_size = 24
com = True  # Center Of Mass
doBck = True
head = 0
tail = 0
criteria = 'EE5'
verbose = True
doPrint = False




In [15]:

butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
rbutler.getKeys('raw')
lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visit, arm=arm) 
calExp = butler.get("calexp", visit=visit, arm=cam[0])

print(f'Start visit {visit} of {cam} with {peaklist}')
print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")


Start visit 24258 of r3 with Imqual_LAM_peaklist_DEBUG_R3_Jan2020_WS_slit.csv
/data/drp/sps/rerun/ws_slit/detrend/


In [16]:
ch.load_np("image", calExp.image.array, 'fits', {})

0

In [ ]:
print(lamps)

In [19]:
canvas = viewer.canvas('Peak')
#canvas = viewer.canvas('B1_DETREND')


In [ ]:
canvas.clear()

In [37]:
peaks = pd.read_csv(peaklist)

In [21]:
for (X,Y), group in peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=20, color='red')
        

In [39]:
peaks[peaks.X>2000]["Y"] = peaks[peaks.X>2000]["Y"] - 10

/software/drp/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
peaks.loc[peaks.X>2000,"Y"] = peaks.loc[peaks.X>2000,"Y"]  - 10

In [47]:
peaks.loc[peaks.X>2000,"Y"] 

0      887.814996
1     1241.434294
2     2021.722183
3     3404.124088
4     4053.937083
5      867.094274
6     1219.346396
7     1997.131356
8     3374.384169
9     4020.156844
10     857.414923
11    1209.116620
12    1985.946328
13    3361.227536
14    4005.386944
Name: Y, dtype: float64

In [49]:
peaks.to_csv("Imqual_LAM_peaklist_DEBUG_R3_Jan2020_WS_slit_2.csv")

In [ ]:
dbutler = dafPersist.Butler('/drp/cluster/repo/')

In [ ]:
dmap = dbutler.get("detectorMap", visit0=3128, spectrograph=1, arm='r', immediate=True)

In [ ]:
for fiber in dmap.getFiberId():
    print(fiber, dmap.getXCenter(fiber, 2000))

In [ ]:
waves = selected_peaks.wavelength.unique()
print(waves)

In [ ]:
dmap.findPoint()

In [ ]:
fiber = 554
wave = waves[1]

In [ ]:
X,Y = dmap.findPoint(fiber, wave)journalctl -f | grep --line-buffered kde

In [ ]:
canvas.add('circle',float(X),float(Y), radius=30, color='blue')


In [ ]:
fibers = [2,102,166,489,554,650]


In [ ]:
for wave in waves:
    for fiber in fibers:
        X,Y = dmap.findPoint(fiber, wave)
        canvas.add('circle',float(X),float(Y)+30, radius=30, color='lightblue')

In [ ]:
new_peak_list = pd.DataFrame(columns = ['fiber','wavelength','element','exptime','arm','X','Y','peak'])

In [ ]:
i = 0
for fiber in fibers:
    p = 1
    for wave in waves:
        X,Y = dmap.findPoint(fiber, wave)
        Y = Y + 30
        new_peak_list.loc[i] = [fiber, wave, "Ar",0, "r",X,Y, p]
        i += 1
        p += 1
    

In [ ]:
new_peak_list

In [ ]:
new_peak_list.to_csv("Imqual_LAM_peaklist_DEBUG_R3_Jan2020_WS_slit.csv")

In [ ]:
canvas = viewer.canvas('Peak')
#canvas = viewer.canvas('B1_DETREND')


In [ ]:
canvas.clear()

In [ ]:
peaks = pd.read_csv(peaklist)

In [ ]:
selected_peaks = peaks[peaks.arm == arm]
selected_peaks = selected_peaks[(selected_peaks.element.str.contains('|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))))]


In [ ]:
selected_peaks

In [ ]:
for (X,Y), group in selected_peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=10, color='red')
        

In [ ]:
for (X,Y), group in peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=20, color='red')
        

In [ ]:
canvas.clear()

In [ ]:
delta_px = -48
delta_py = +10

In [ ]:
#Neon
delta_px = +45
delta_py = +28

In [ ]:
#R2 HgAr
delta_px = +41
delta_py = +17

In [ ]:
new_peaks = selected_peaks
new_peaks["X"] = new_peaks.apply(lambda x : x["X"] + delta_px, axis=1)
new_peaks["Y"] = new_peaks.apply(lambda x : x["Y"] + delta_py, axis=1)

In [ ]:
for (X,Y), group in new_peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=30, color='orange')
        

In [ ]:
fibers = new_peaks.fiber.unique()
print(fibers)

In [ ]:
del_fiber = fibers[8]
print(del_fiber)

In [ ]:
new_peaks = new_peaks[new_peaks.fiber != del_fiber]

In [ ]:
new_peaks.fiber.unique()

In [ ]:
for (X,Y), group in new_peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=30, color='blue')
        

In [ ]:
waves = new_peaks.wavelength.unique()
print(waves)

In [ ]:
del_wave = waves[3]
print(del_wave)

In [ ]:
new_peaks = new_peaks[new_peaks.wavelength != del_wave]

In [ ]:
keep_wave = waves[-1]
print(keep_wave)

In [ ]:
new_peaks = new_peaks[new_peaks.wavelength == keep_wave]

In [ ]:
canvas.clear()

In [ ]:
for (X,Y), group in new_peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=30, color='lightblue')
        

In [ ]:
new_peaks.loc[(new_peaks.fiber == 339), 'fiber'] = 315

In [ ]:
new_peaks.loc[(new_peaks.fiber == 315), 'X'] = 2178

In [ ]:
new_peaks.to_csv("Imqual_LAM_peaklist_DEBUG_HgAr_R2_Dec2020.csv")

In [ ]:
new_peaks = peaks
new_peaks["X"] = new_peaks.apply(lambda x : x["X"] + delta_px, axis=1)
new_peaks["Y"] = new_peaks.apply(lambda x : x["Y"] + delta_py, axis=1)

In [ ]:
new_peaks.to_csv("Imqual_LAM_peaklist_DEBUG_B2_Dec2020.csv")

In [ ]:
hgar = pd.read_csv("Imqual_LAM_peaklist_DEBUG_HgAr_3peaks_B2_Dec2020.csv")
neon = pd.read_csv("Imqual_LAM_peaklist_DEBUG_Neon_3peaks_B2_Dec2020.csv")

In [ ]:
peaklist_debugb2 = pd.concat([hgar,neon])

In [ ]:
peaklist_debugb2

In [ ]:
peaklist_debugb2.drop(columns="Unnamed: 0", axis=1, inplace=True)

In [ ]:
peaklist_debugb2.to_csv("Imqual_LAM_peaklist_DEBUG_B2_9peaks_Dec2020.csv")

In [ ]:
shell.add_channel('flat')

In [ ]:
experimentId = 705
filelist = constructFilelist(experimentId, rerun='ginga')
filepath = filelist.filepath[0]

In [ ]:
from astropy.io import fits
hdulist = fits.open(filepath, "readonly")
ch.load_hdu("r", hdulist, 1)

In [ ]:
peaklist = "/home/pfs/dev/ait-notebook/optical/input/hgar_peaks_exptime_dec2017.csv"


In [ ]:
peak = pd.read_csv(peaklist)

In [ ]:
canvas = ch.add_canvas()


In [ ]:
# Draw circles around all objects
Circle = canvas.get_draw_class('circle')
for (X,Y), group in peak.groupby(["X","Y"]):
#    x, y = x1+obj.objx, y1+obj.objy
    x = X
    y = Y

    canvas.add(Circle(x, y, radius=10, color='yellow'))